In [1]:
import numpy as np
import strawberryfields as sf
import tensorflow as tf
import random

In [2]:
tf_displacement_magnitude = tf.Variable(0.1)

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
steps = 500

batch_size = 10
threshold = 0.5

for step in range(steps):
    batch = [1 if random.random() > threshold else -1 for _ in range(batch_size)]
    alpha_val = 0.7 * np.array(batch)
    
    p_1 = batch.count(1)/batch_size
    p_0 = 1 - p_1
        
    eng = sf.Engine(backend="tf", backend_options={
        "cutoff_dim": 7,
        "batch_size": batch_size,
    })
    
    circuit = sf.Program(1)

    displacement_magnitude = circuit.params("displacement_magnitude")
    
    alpha = circuit.params("alpha")

    with circuit.context as q:
        sf.ops.Dgate(alpha, 0.0) | q[0]
        sf.ops.Dgate(displacement_magnitude, 0.0) | q[0]
        
    with tf.GradientTape() as tape:
        results = eng.run(circuit, args={
            "displacement_magnitude": tf_displacement_magnitude,
            "alpha": alpha_val
        })
        
        # get the probability of |0>
        p_zero = results.state.fock_prob([0])
        
        # get the porbability of anything by |0>
        p_one = 1 - p_zero
        
        loss = 0
        
        # p(a|1) = [p(1|a) p(1)] / p(a)
        # p(-a|0) = [p(0|-a) p(0)] / p(-a)
        
        for i, mult in enumerate(batch):
            if mult == 1:
                loss += - (p_one[i] * p_1) / 2
            else:
                loss += - (p_zero[i] * p_0) / 2
                
        loss /= batch_size
        
    gradients = tape.gradient(loss, [tf_displacement_magnitude])
    opt.apply_gradients(zip(gradients, [tf_displacement_magnitude]))
    
    if (step + 1) % 10 == 0:
        print("Learned displacement value at step {}: {}".format(step+1, tf_displacement_magnitude.numpy()))

2021-10-15 14:48:27.467286: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-15 14:48:27.472439: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-15 14:48:27.472943: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-15 14:48:27.473721: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Learned displacement value at step 10: 0.19944506883621216
Learned displacement value at step 20: 0.29802724719047546
Learned displacement value at step 30: 0.3932643532752991
Learned displacement value at step 40: 0.48026978969573975
Learned displacement value at step 50: 0.5585140585899353
Learned displacement value at step 60: 0.6269962787628174
Learned displacement value at step 70: 0.6847254037857056
Learned displacement value at step 80: 0.7314314246177673
Learned displacement value at step 90: 0.7733591198921204
Learned displacement value at step 100: 0.8051719665527344
Learned displacement value at step 110: 0.8300434350967407
Learned displacement value at step 120: 0.8469008207321167
Learned displacement value at step 130: 0.8509359955787659
Learned displacement value at step 140: 0.8581244945526123
Learned displacement value at step 150: 0.8601717352867126
Learned displacement value at step 160: 0.8630684018135071
Learned displacement value at step 170: 0.8592459559440613
Lea